## Установка pandas

In [1]:
### !pip3 install pandas

In [2]:
import pandas as pd

## Чтение данных и фильтрация таблиц

In [26]:
user = pd.read_csv('user.csv', parse_dates=['birthday', 'last_visited'])

In [28]:
user.head()

,id,gender,birthday,grade,region,last_visited
0,1,М,2007-02-11,9.0,Город Санкт-Петербург,2023-08-23 03:04:14
1,2,Ж,2009-01-29,8.0,Город Москва,2023-07-09 09:21:35
2,3,М,2007-09-05,NaN,Город Санкт-Петербург,2023-08-23 18:32:18
3,4,Ж,2007-09-27,9.0,Город Москва,2023-08-18 07:34:11
4,5,Ж,2007-04-08,NaN,Город Москва,2023-02-09 18:31:30


In [27]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1000 non-null   int64         
 1   gender        1000 non-null   object        
 2   birthday      1000 non-null   datetime64[ns]
 3   grade         831 non-null    float64       
 4   region        1000 non-null   object        
 5   last_visited  1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 47.0+ KB


### Задание 1

Сколько в таблице `user` представлено учеников, родившихся в сентябре?

## НЕВЕРНО!!!

In [73]:
(user.birthday.dt.month == 9).sum()

80

### Задание 2

Какое количество учеников в последний раз посещали платформу в первые $14$ дней сентября?

## НЕВЕРНО!!!

In [74]:
((user.last_visited.dt.month == 9) & (user.last_visited.dt.day.between(1, 14))).sum()

29

In [60]:
user[(user.last_visited.dt.month == 9) & (user.last_visited.dt.day.between(1, 14))]

,id,gender,birthday,grade,region,last_visited
47,48,Ж,2007-03-08,NaN,Новосибирская область,2023-09-09 06:29:41
74,75,М,2009-05-02,8.0,Город Москва,2023-09-09 07:34:33
92,93,М,2010-01-15,7.0,Город Москва,2023-09-09 18:14:03
117,118,Ж,2007-09-14,NaN,Город Санкт-Петербург,2023-09-09 17:27:50
120,121,М,2008-11-06,9.0,Город Санкт-Петербург,2023-09-09 10:02:10
128,129,М,2010-01-31,7.0,Город Москва,2023-09-09 15:09:37
131,132,М,2008-07-22,9.0,Новосибирская область,2023-09-09 16:41:39
205,206,Ж,2008-11-24,8.0,Нижегородская область,2023-09-09 03:13:05
232,233,М,2009-04-18,8.0,Город Москва,2023-09-09 03:01:12
279,280,М,2010-01-08,7.0,Город Санкт-Петербург,2023-09-09 13:04:01


In [70]:
user.sort_values('last_visited', ascending=False)[83:114]

,id,gender,birthday,grade,region,last_visited
36,37,Ж,2008-04-10,NaN,Нижегородская область,2023-10-09 02:28:13
434,435,М,2009-06-24,NaN,Город Москва,2023-09-09 22:08:06
570,571,М,2006-05-11,10.0,Новосибирская область,2023-09-09 21:23:00
561,562,Ж,2007-11-10,9.0,Город Москва,2023-09-09 21:22:47
411,412,Ж,2008-12-21,NaN,Город Санкт-Петербург,2023-09-09 18:56:59
868,869,М,2007-01-04,10.0,Город Санкт-Петербург,2023-09-09 18:35:47
92,93,М,2010-01-15,7.0,Город Москва,2023-09-09 18:14:03
117,118,Ж,2007-09-14,NaN,Город Санкт-Петербург,2023-09-09 17:27:50
131,132,М,2008-07-22,9.0,Новосибирская область,2023-09-09 16:41:39
876,877,Ж,2009-12-30,NaN,Краснодарский край,2023-09-09 15:22:05


In [77]:
user.last_visited.dt.date.unique()

array([datetime.date(2023, 8, 23), datetime.date(2023, 7, 9),
       datetime.date(2023, 8, 18), datetime.date(2023, 2, 9),
       datetime.date(2023, 3, 9), datetime.date(2023, 8, 22),
       datetime.date(2023, 8, 21), datetime.date(2023, 5, 9),
       datetime.date(2023, 10, 9), datetime.date(2023, 8, 9),
       datetime.date(2023, 4, 9), datetime.date(2023, 11, 9),
       datetime.date(2023, 8, 31), datetime.date(2023, 8, 20),
       datetime.date(2023, 8, 16), datetime.date(2023, 12, 9),
       datetime.date(2023, 8, 14), datetime.date(2023, 1, 9),
       datetime.date(2023, 8, 28), datetime.date(2023, 8, 29),
       datetime.date(2023, 8, 25), datetime.date(2023, 6, 9),
       datetime.date(2023, 8, 30), datetime.date(2023, 8, 26),
       datetime.date(2023, 8, 17), datetime.date(2023, 9, 9),
       datetime.date(2023, 8, 15), datetime.date(2023, 8, 24),
       datetime.date(2023, 8, 19), datetime.date(2023, 8, 27),
       datetime.date(2023, 8, 13)], dtype=object)

### Задание 3

На сколько в данных больше девятиклассников, чем девятиклассниц?

In [41]:
((user.gender == 'М') & (user.grade == 9)).sum() - ((user.gender == 'Ж') & (user.grade == 9)).sum()

5

### Задание 4

Какой процент учеников, у которых указан класс, обучается в $9$-м классе?

Ответ округлите до целых.

In [45]:
((user.grade == 9).sum() / user.grade.count() * 100).round()

24.0

### Задание 5

Сколько в данных представлено учеников из регионов, в названиях которых в качестве подстроки содержится слово «город»?

При расчёте статистики приведите названия регионов к строчному написанию.

In [72]:
user.region.str.lower().str.contains('город').sum()
# user[(user.region.str.contains('город')) & (user.gender == 'М')].shape

765

## Группировка, агрегация и соединение таблиц

In [ ]:
course = pd.read_csv('course.csv')

In [ ]:
user_course_progress = pd.read_csv('user_course_progress.csv')

**Относительным прогрессом** будем называть отношение суммарного числа баллов, которые ученик набрал за курс (колонка `progress` в таблице `user_course_progress`), к максимальному количеству баллов, которые ученик в принципе мог набрать за данный курс (колонка `max_progress` в таблице `course`).

С помощью представленных данных найдите ответ на следующий вопрос: в курсах какого предмета больше всего учеников, чей относительный прогресс не меньше $0.8$?

В рамках следующих заданий вам будет предложено, действуя поэтапно, найти ответ на этот вопрос.

### Задание 1

Произведите соединение таблиц `course` и `user_course_progress`.

В качестве ответа укажите количество строчек итоговой таблицы, в которых представлены ученики курсов по химии.

### Задание 2

Обратите внимание, что у части курсов нет значения в колонке `max_score`. Уберите из полученной в предыдщуем задании таблицы все строчки, в которых пропущено значение колонки `max_score`.

В качестве ответа укажите количество строчек в итоговой таблице.

### Задание 3

Добавьте в полученную в предыдущем задании таблицу колонку `relative_progress`, в которой для каждого ученика будет храниться значение его относительного прогресса в конкретном курсе.

В качестве ответа укажите среднее значение относительного прогресса в таблице. Ответ округлите до двух знаков после запятой с помощью функции `round`, которая встроена в Series.

### Задание 4

Оставьте в полученной в предыдущем задании таблице только те строчки, в которых значение относительного прогресса не меньше $0.8$.

В качестве ответа укажите число строчек в получившейся таблице.

### Задание 5

Произведите группировку строчек таблицы по предметам и посчитайте, сколько учеников с относительным прогрессом не меньше $0.8$ оказалось в каждом из предметов.

В качестве ответа заполните таблицу, в которой каждому предмету сопоставлено число таких учеников.

Если один и тот же ученик учился в разных курсах одного и того же предмета и в каждом из этих курсов достиг относительного прогресса не менее $0.8$, при подсчёте учтите его несколько раз.

### Задание 6

В курсах какого предмета больше всего учеников, достигших относительного прогресса не менее $0.8$?